# Extracting the data using web driver and applying search engine

In [36]:
import selenium
import pandas as pd
from bs4 import BeautifulSoup
import time

In [37]:
from selenium.webdriver import Chrome, Firefox
PATH= "C:\webdriver\geckodriver.exe"

In [38]:
browser=Firefox()
browser.get("https://appsrv.pace.edu/ScheduleExplorerLive/")

In [39]:
from selenium.webdriver.support.ui import Select

In [40]:
term=Select(browser.find_element_by_id('checkterm'))
term.select_by_visible_text('Fall 2020')

In [41]:
level=Select(browser.find_element_by_id('level'))
level.select_by_visible_text('Graduate')

In [42]:
subject=Select(browser.find_element_by_id('subject'))
subjects= [subject.text for subject in subject.options]

In [43]:
IS_text=[subject for subject in subjects if 'IS' in subject][0]
subject.select_by_visible_text(IS_text)
search_button=browser.find_element_by_id('submitbutton')
search_button.click()

In [44]:
## Extra to increase the table size
dd_number=Select(browser.find_element_by_name('mainResultsTable_length'))
dd_number.select_by_value('100')


In [45]:
html=browser.page_source
courses=pd.read_html(html)[0]
soup=BeautifulSoup(html,'html.parser')
links=soup.find_all('a',{'href':True})

## I will only get the links for CRN numbers

links_selected=soup.find_all('a',{'data-toggle':'modal'})
crn_links=[link for link in links_selected  if link.next.strip().isnumeric()]

In [46]:
def read_crn(crn_link):
    link=browser.find_element_by_partial_link_text(crn_link.text.strip())
    time.sleep(.5)
    link.click()
    popup_content=browser.find_element_by_id('modalBodyContentDiv')
    
    df=pd.read_html(popup_content.get_attribute('innerHTML'))[0].T
    df=pd.read_html(popup_content.get_attribute('innerHTML'))[0].T
    header=df.iloc[0]
    df=df[1:]
    df.columns=header
   
    pop_up_header=browser.find_element_by_class_name('modal-header')
    header_close=pop_up_header.find_element_by_tag_name('Button')
    time.sleep(.5)
    header_close.click()
    
    return df

In [48]:
dfs=[]
for crn_link in crn_links:
    df=read_crn(crn_link)
    dfs.append(df)

In [49]:
all_is_cours_decs=pd.concat(dfs)

In [50]:
all_is_cours_decs['CRN:']=pd.to_numeric(all_is_cours_decs['CRN:'])

In [51]:
main_table=pd.read_html(html)[0]

In [52]:
main_table.head()

,CRN,Subject,Course#,Title,Sched Type,Credits,Campus,Instructional Method*,Section Comments,Days**,Time,Capacity,Seats Avail.,Instructor,More Info
0,73592,IS,522,Advanced Spreadsheet Skills,LEC,3,Online,Asynchronous (ASYNC),NaN,NaN,NaN,25,18,Henry Gaylord,More Info
1,70426,IS,612,Introduction to Coding,LEC,3,NYC,Web-assisted (WA),NaN,W,06:10pm-09:00pm,23,03,Dhruvil Gandhi,More Info
2,70376,IS,613,Database Management Systems,LEC,3,NYC,Web-assisted (WA),NaN,T,06:10pm-09:00pm,30,CLOSED,Nathifa Lewis,Prerequisites
3,71721,IS,613,Database Management Systems,LEC,3,Online,Synchronous (SYNC),NaN,T,06:10pm-09:00pm,30,01,Helen Uzamere,Prerequisites
4,72779,IS,613,Database Management Systems,LEC,3,Online,Mixed (MIXAS),NaN,S,09:00am-12:00pm,30,14,Paul Dantzig,Prerequisites


In [53]:
all_is_cours_decs=all_is_cours_decs.rename({'CRN:':'CRN'}, axis='columns')
all_is_cours_decs.head()

,COURSE TITLE:,SUBJECT/COURSE#:,CRN,CREDIT RANGE:,SCHOOL:,DEPARTMENT:,LEVEL:,COREQUISITE:,PREREQUISITE:,DESCRIPTION:
1,Advanced Spreadsheet Skills,IS 522,73592,3.000,Seidenberg School of CSIS,Internet Technology,Graduate,None,None,Course Description: This is an online course t...
1,Introduction to Coding,IS 612,70426,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Course Description: This course provides an in...
1,Database Management Systems,IS 613,70376,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...
1,Database Management Systems,IS 613,71721,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...
1,Database Management Systems,IS 613,72779,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...


In [112]:
combined_IS=main_table.merge(all_is_cours_decs, how='left', on='CRN')
combined_IS.columns

Index(['CRN', 'Subject', 'Course#', 'Title', 'Sched Type', 'Credits', 'Campus',
       'Instructional Method*', 'Section Comments', 'Days**', 'Time',
       'Capacity', 'Seats Avail.', 'Instructor', 'More Info', 'COURSE TITLE:',
       'SUBJECT/COURSE#:', 'CREDIT RANGE:', 'SCHOOL:', 'DEPARTMENT:', 'LEVEL:',
       'COREQUISITE:', 'PREREQUISITE:', 'DESCRIPTION:'],
      dtype='object')

In [115]:
combined_IS


,CRN,Subject,Course#,Title,Sched Type,Credits,Campus,Instructional Method*,Section Comments,Days**,...,More Info,COURSE TITLE:,SUBJECT/COURSE#:,CREDIT RANGE:,SCHOOL:,DEPARTMENT:,LEVEL:,COREQUISITE:,PREREQUISITE:,DESCRIPTION:
0,73592,IS,522,Advanced Spreadsheet Skills,LEC,3,Online,Asynchronous (ASYNC),NaN,NaN,...,More Info,Advanced Spreadsheet Skills,IS 522,3.000,Seidenberg School of CSIS,Internet Technology,Graduate,None,None,Course Description: This is an online course t...
1,70426,IS,612,Introduction to Coding,LEC,3,NYC,Web-assisted (WA),NaN,W,...,More Info,Introduction to Coding,IS 612,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Course Description: This course provides an in...
2,70376,IS,613,Database Management Systems,LEC,3,NYC,Web-assisted (WA),NaN,T,...,Prerequisites,Database Management Systems,IS 613,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...
3,71721,IS,613,Database Management Systems,LEC,3,Online,Synchronous (SYNC),NaN,T,...,Prerequisites,Database Management Systems,IS 613,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...
4,72779,IS,613,Database Management Systems,LEC,3,Online,Mixed (MIXAS),NaN,S,...,Prerequisites,Database Management Systems,IS 613,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,Pre-requisite for IS 613 ( Course : IS 520 . M...,Course Description: This course focuses on the...
5,70063,IS,617,Information Systems Principles,LEC,3,Online,Asynchronous (ASYNC),NaN,NaN,...,More Info,Information Systems Principles,IS 617,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Course Description: This course examines manag...
6,71371,IS,620,IS and Organizational Strategy,LEC,3,NYC,HyFlex (HYFLX),NaN,W,...,More Info,Information Systems and Organizational Strategy,IS 620,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Course Description: This course brings to life...
7,70377,IS,623,Info Sys Design & Devlpmt,LEC,3,NYC,HyFlex (HYFLX),NaN,M,...,More Info,Information Systems Design and Development,IS 623,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Not open to students who have taken IS 621. Co...
8,71427,IS,623,Info Sytems Design & Devlpmt,LEC,3,Online,Asynchronous (ASYNC),NaN,NaN,...,More Info,Information Systems Design and Development,IS 623,3.000,Seidenberg School of CSIS,Computer Information Technlgy,Graduate,None,None,Not open to students who have taken IS 621. Co...
9,72669,IS,628,Research Mthds for User Exp.,LEC,3,Online,Mixed (MIXAS),"Meeting 163 William St, Rm 238",W,...,More Info,Research Methods for User Experience,IS 628,3.000,Seidenberg School of CSIS,Information Systems,Graduate,None,None,"Course Description: In User Experience, the fo..."


In [182]:
import pandas as pd 
df = pd.DataFrame(combined_IS,columns=['CRN','Title','SUBJECT/COURSE#:','DESCRIPTION:'])
df.rename(columns = {'Title': 'TITLE', 'SUBJECT/COURSE#:':'SUBJECT_COURSE', 'DESCRIPTION:':'DESCRIPTION'},  
          inplace = True)
df['DESCRIPTION'] = df['DESCRIPTION'].str.replace('Course Description:',' ')
df

,CRN,TITLE,SUBJECT_COURSE,DESCRIPTION
0,73592,Advanced Spreadsheet Skills,IS 522,This is an online course that introduces stu...
1,70426,Introduction to Coding,IS 612,This course provides an introduction to prog...
2,70376,Database Management Systems,IS 613,This course focuses on the theoretical and p...
3,71721,Database Management Systems,IS 613,This course focuses on the theoretical and p...
4,72779,Database Management Systems,IS 613,This course focuses on the theoretical and p...
5,70063,Information Systems Principles,IS 617,This course examines managerial information ...
6,71371,IS and Organizational Strategy,IS 620,This course brings to life the latest busine...
7,70377,Info Sys Design & Devlpmt,IS 623,Not open to students who have taken IS 621. ...
8,71427,Info Sytems Design & Devlpmt,IS 623,Not open to students who have taken IS 621. ...
9,72669,Research Mthds for User Exp.,IS 628,"In User Experience, the foundation is resear..."


In [183]:
df = df.dropna()

In [184]:
df

,CRN,TITLE,SUBJECT_COURSE,DESCRIPTION
0,73592,Advanced Spreadsheet Skills,IS 522,This is an online course that introduces stu...
1,70426,Introduction to Coding,IS 612,This course provides an introduction to prog...
2,70376,Database Management Systems,IS 613,This course focuses on the theoretical and p...
3,71721,Database Management Systems,IS 613,This course focuses on the theoretical and p...
4,72779,Database Management Systems,IS 613,This course focuses on the theoretical and p...
5,70063,Information Systems Principles,IS 617,This course examines managerial information ...
6,71371,IS and Organizational Strategy,IS 620,This course brings to life the latest busine...
7,70377,Info Sys Design & Devlpmt,IS 623,Not open to students who have taken IS 621. ...
8,71427,Info Sytems Design & Devlpmt,IS 623,Not open to students who have taken IS 621. ...
9,72669,Research Mthds for User Exp.,IS 628,"In User Experience, the foundation is resear..."


In [185]:
import nltk
stopword = nltk.corpus.stopwords.words('english')
stopword

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [186]:
import re
wn = nltk.WordNetLemmatizer()
def clean(text):
    text=re.sub(r'[^\w\s]',' ',text).lower()   ### Remove punctuation by removing non-space alpha-num characters
    words=re.split(r'\W+', text)                   ### Split by remaining non-alphanumeric characters 
    non_stop_words=[word for word in words if word not in stopword]        ### Remove stopwords
    lem_words= [wn.lemmatize(word) for word in non_stop_words]             ### Clean text ==
    clean_text= ' '.join(lem_words)
    return clean_text

In [187]:
df['clean_text'] = df['DESCRIPTION'].apply(clean)
df

,CRN,TITLE,SUBJECT_COURSE,DESCRIPTION,clean_text
0,73592,Advanced Spreadsheet Skills,IS 522,This is an online course that introduces stu...,online course introduces student feature micr...
1,70426,Introduction to Coding,IS 612,This course provides an introduction to prog...,course provides introduction programming pyth...
2,70376,Database Management Systems,IS 613,This course focuses on the theoretical and p...,course focus theoretical practical aspect fil...
3,71721,Database Management Systems,IS 613,This course focuses on the theoretical and p...,course focus theoretical practical aspect fil...
4,72779,Database Management Systems,IS 613,This course focuses on the theoretical and p...,course focus theoretical practical aspect fil...
5,70063,Information Systems Principles,IS 617,This course examines managerial information ...,course examines managerial information requir...
6,71371,IS and Organizational Strategy,IS 620,This course brings to life the latest busine...,course brings life latest business research c...
7,70377,Info Sys Design & Devlpmt,IS 623,Not open to students who have taken IS 621. ...,open student taken 621 course provides introdu...
8,71427,Info Sytems Design & Devlpmt,IS 623,Not open to students who have taken IS 621. ...,open student taken 621 course provides introdu...
9,72669,Research Mthds for User Exp.,IS 628,"In User Experience, the foundation is resear...",user experience foundation research required ...


In [188]:
from sklearn.feature_extraction.text import CountVectorizer

In [189]:
count_vect=CountVectorizer()

In [190]:
X_counts=count_vect.fit_transform(df['clean_text'])

In [191]:
X_counts.toarray()

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 1, 0]], dtype=int64)

In [192]:
vocab=count_vect.get_feature_names()


In [193]:
vectorset=pd.DataFrame(X_counts.toarray(), columns=vocab)
vectorset

,2016,24,27000,27001,27002,600,621,695q,access,accessing,...,wide,widening,within,without,work,workbook,worksheet,world,written,yet
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,1,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [194]:
top20wordslist = vectorset.sum().nlargest(20)
top20wordslist


data           79
course         65
system         48
management     45
student        34
information    33
security       26
database       22
project        22
business       19
development    19
topic          19
application    18
design         17
rotation       17
include        16
plv            15
also           14
software       14
spring         14
dtype: int64

In [195]:
description = df['DESCRIPTION'].agg(len).nlargest(1)
largestdescription = description.idxmax()
df.iloc[largestdescription]


CRN                                                           72455
TITLE                                Big Data & Information Systems
SUBJECT_COURSE                                               IS 669
DESCRIPTION         Data and analytics are changing the world an...
clean_text         data analytics changing world way making deci...
Name: 25, dtype: object

In [196]:
word_count = vectorset.iloc[largestdescription].sum()
word_count

160